In [2]:
# code_15_BERT_PROPN.py
# !pip install transformers
import os

print(os.getcwd())
# GAP dataset: https://github.com/google-research-datasets/gap-coreference

D:\GitProjects\pytorch-GNN\pytorch-GNN-2nd\notebook


In [3]:

#提取代词特征

import pandas as pd
import pickle
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, BertConfig


In [3]:
# BERT_PATH = '/home/xhm/nlp/pytorch-GNN/pytorch-GNN-2nd/huggingface/bert-base-uncased'

# tokenizer = BertTokenizer.from_pretrained(BERT_PATH)

# print(tokenizer.tokenize('I have a good time, thank you.'))

# bert = BertModel.from_pretrained(BERT_PATH)

# print('load bert model over')


In [4]:

#指定设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#读取数据    
# 变量命名有问题？
df_test = pd.read_csv("../gap-coreference/gap-development.tsv", delimiter="\t")
df_train_val = pd.concat([
    pd.read_csv("../gap-coreference/gap-test.tsv", delimiter="\t"),
    pd.read_csv("../gap-coreference/gap-validation.tsv", delimiter="\t")
], axis=0)



cpu


In [14]:

def getmodel():
    #加载词表文件tokenizer
    tokenizer = BertTokenizer.from_pretrained('../huggingface/bert-base-uncased')

    #添加特殊词
    special_tokens_dict = {'additional_special_tokens': ["[THISISA]", "[THISISB]", "[THISISP]"]}
    tokenizer.add_special_tokens(special_tokens_dict)  #添加特殊词
    print('from getmodel', tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids)

    model = BertModel.from_pretrained('../huggingface/bert-base-uncased')  #加载模型
    return tokenizer, model


############################


def insert_tag(row, hasbrack=True):
    # P: 目标代词   A: 候选名称1    B: 候选名称2
    orgtag = [" [THISISA] ", " [THISISB] ", " [THISISP] "]
    if hasbrack == False:
        orgtag = [" THISISA ", " THISISB ", " THISISP "]

    #按照插入的位置，从大到小排序[(383, ' THISISP '), (366, ' THISISB '), (352, ' THISISA ')]
    to_be_inserted = sorted([
        (row["A-offset"], orgtag[0]),
        (row["B-offset"], orgtag[1]),
        (row["Pronoun-offset"], orgtag[2])], key=lambda x: x[0], reverse=True)

    text = row["Text"]  #len 443
    for offset, tag in to_be_inserted:  #先插最后的，不会影响前面
        text = text[:offset] + tag + text[offset:]  #（插到每个代词的前面）
    return text  #len 470 (443+3*9)


# 将标签分离，并返特殊标签偏移位置
def tokenize(sequence_ind, tokenizer, sequence_mask=None):  #将标签分离，并返回标签偏移位置
    entries = {}
    final_tokens = []
    final_mask = []

    for i, one in enumerate(sequence_ind):
        if one in tokenizer.additional_special_tokens_ids:
            tokenstr = tokenizer.convert_ids_to_tokens(one)
            entries[tokenstr] = len(final_tokens)
            continue
        final_tokens.append(one)
        if sequence_mask is not None:
            final_mask.append(sequence_mask[i])
    return final_tokens, (entries["[THISISA]"], entries["[THISISB]"], entries["[THISISP]"]), final_mask


def savepkl(df, name):
    bert_prediction = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        #循环内部
        text = insert_tag(row)  #插入标签
        sequence_ind = tokenizer.encode(text)  #向量化
        tokens, offsets, _ = tokenize(sequence_ind, tokenizer)  #获取标签偏移
        '''
        [CLS ... SEP]
        [101, 11199, 10093, 3877, 1011, 1011, 2209, 1996, 2610, 2961, 6513, 1997, 4079, 1010, 8538, 1012, 14019, 2011, 4079, 1999, 1996, 2345, 2792, 1997, 2186, 1015, 1010, 2044, 2002, 7771, 2007, 8437, 1010, 1998, 2003, 2025, 2464, 2153, 1012, 18188, 2726, 2209, 19431, 13737, 1010, 15595, 1005, 1055, 2767, 1998, 2036, 1037, 2095, 2340, 11136, 1999, 4079, 1005, 1055, 2465, 1012, 14019, 2014, 6898, 2206, 4079, 1005, 1055, 6040, 2044, 2002, 2876, 1005, 1056, 2031, 3348, 2007, 2014, 2021, 2101, 11323, 2023, 2001, 2349, 2000, 2032, 9105, 26076, 2125, 2014, 2767, 15595, 1012, 102]
        (42, 45, 62)
        '''
        print(tokens, offsets)
        token_tensor = torch.LongTensor([tokens]).to(device)
        #         bert_outputs,bert_last_outputs=  model(token_tensor)  #[1, 107, 768] , [1, 768]
        #         print(type(model))
        #         <class 'str'> last_hidden_state
        #         print(type(bert_outputs), bert_outputs)
        #         print(type(bert_last_outputs), bert_last_outputs)
        res = model(token_tensor)
        # odict_keys(['last_hidden_state', 'pooler_output'])
        # print(res.keys())
        bert_last_hidden_state, bert_pooler_output = res[:2]  #[1, 107, 768] , [1, 768]
        # print(bert_last_hidden_state.shape, bert_pooler_output.shape)
        extracted_outputs = bert_last_hidden_state[:, offsets, :]  #根据偏移位置抽取单词的特征向量
        bert_prediction.append(extracted_outputs.cpu().numpy())
    pickle.dump(bert_prediction, open(name, "wb"))



In [ ]:
'''
在使用 BERT 模型时，你可以根据任务需求选择使用 'last_hidden_state' 或 'pooler_output'，或者结合两者进行下游任务的建模。
例如，在一些任务中，可以使用 'pooler_output' 作为整个句子的表示，而在其他任务中，可以使用 'last_hidden_state' 来获取每个单词的表示。
'''

In [15]:

if __name__ == '__main__':
    tokenizer, model = getmodel()
    model.to(device)
    # 在这个代码块内关闭梯度计算，从而节省计算资源和内存。
    torch.set_grad_enabled(False)
    
    # 测试集的P、A、B这些专有名词的特征向量
    savepkl(df_test, 'output/test_bert_outputs_forPROPN.pkl')
    savepkl(df_train_val, 'output/bert_outputs_forPROPN.pkl')

from getmodel ['[THISISA]', '[THISISB]', '[THISISP]'] [30522, 30523, 30524]


Some weights of the model checkpoint at ../huggingface/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 1/2000 [00:00<05:07,  6.50it/s]

[101, 11199, 10093, 3877, 1011, 1011, 2209, 1996, 2610, 2961, 6513, 1997, 4079, 1010, 8538, 1012, 14019, 2011, 4079, 1999, 1996, 2345, 2792, 1997, 2186, 1015, 1010, 2044, 2002, 7771, 2007, 8437, 1010, 1998, 2003, 2025, 2464, 2153, 1012, 18188, 2726, 2209, 19431, 13737, 1010, 15595, 1005, 1055, 2767, 1998, 2036, 1037, 2095, 2340, 11136, 1999, 4079, 1005, 1055, 2465, 1012, 14019, 2014, 6898, 2206, 4079, 1005, 1055, 6040, 2044, 2002, 2876, 1005, 1056, 2031, 3348, 2007, 2014, 2021, 2101, 11323, 2023, 2001, 2349, 2000, 2032, 9105, 26076, 2125, 2014, 2767, 15595, 1012, 102] (42, 45, 62)
[101, 2002, 3473, 2039, 1999, 6473, 2669, 1010, 4307, 1996, 2117, 4587, 1997, 2274, 2336, 2164, 2010, 3428, 1010, 5965, 1998, 5146, 1998, 5208, 1010, 25532, 1006, 27233, 7685, 1007, 1998, 14749, 1012, 2010, 2152, 2082, 2420, 2020, 2985, 2012, 2047, 25487, 2152, 2082, 1999, 2663, 7159, 2912, 1010, 4307, 1012, 11407, 3273, 2007, 6795, 24520, 2013, 4085, 2000, 3999, 1012, 2010, 3722, 1010, 5217, 1011, 6908, 8360

  0%|          | 3/2000 [00:00<05:34,  5.97it/s]

[101, 2002, 2018, 2042, 20847, 2000, 3519, 1010, 2021, 5295, 1999, 2901, 2000, 5138, 1037, 2695, 2004, 6059, 2000, 4380, 1012, 2139, 2474, 2061, 2696, 2153, 2743, 2005, 3099, 1997, 1039, 1008, 16428, 16429, 2050, 1999, 2889, 1012, 3249, 2011, 3099, 12262, 2480, 2011, 2058, 2321, 1003, 1010, 2023, 3732, 2275, 5963, 2001, 3278, 2138, 2009, 3465, 2139, 2474, 2061, 2696, 2172, 1997, 2010, 2490, 2306, 1996, 2074, 24108, 9863, 2283, 1006, 2029, 2001, 13862, 2007, 3377, 1999, 1996, 2889, 3054, 1011, 3408, 1007, 1010, 2877, 2000, 2343, 5828, 2273, 6633, 1005, 1055, 20380, 1997, 1037, 3584, 2283, 2862, 1999, 1039, 1008, 16428, 16429, 2050, 2005, 1996, 2857, 3054, 1011, 2744, 3864, 1010, 1998, 2000, 2139, 2474, 2061, 2696, 1005, 1055, 4945, 2000, 12452, 1037, 2835, 1999, 3519, 1012, 102] (41, 57, 63)


  0%|          | 4/2000 [00:00<05:23,  6.17it/s]

[101, 1996, 2783, 2372, 1997, 4126, 2031, 2036, 2864, 1999, 2624, 3799, 2104, 1996, 2316, 2171, 1005, 1005, 6556, 7193, 1036, 1036, 1012, 4894, 2038, 2405, 2048, 2573, 1997, 4349, 1999, 3522, 2086, 1024, 8831, 1997, 3109, 1010, 2029, 2003, 3205, 1999, 1996, 2600, 1998, 4897, 2534, 1997, 4476, 3075, 1010, 1998, 1037, 5189, 20364, 2614, 2234, 2013, 2682, 1012, 4635, 2038, 2550, 3365, 3152, 1006, 2104, 2010, 2613, 2171, 1010, 2888, 29062, 1007, 2164, 1996, 2718, 1996, 6548, 1998, 3817, 10773, 1012, 102] (36, 71, 67)
[101, 2014, 4203, 10768, 3850, 2834, 1999, 2384, 2001, 2004, 27617, 2401, 1999, 1996, 8001, 3179, 1997, 2175, 3669, 5558, 2615, 1005, 1055, 7110, 8447, 7849, 1012, 2016, 6369, 2006, 1996, 4745, 11605, 13250, 5302, 20846, 3405, 1997, 1996, 3850, 1012, 2005, 2010, 3850, 3460, 9593, 1010, 5922, 2128, 13088, 12184, 1996, 2535, 1997, 14433, 6728, 11837, 18826, 1010, 2761, 1037, 2033, 12036, 1011, 10430, 2535, 1010, 2005, 10430, 2376, 1010, 1998, 14043, 6369, 1996, 2128, 15773, 2112

  0%|          | 4/2000 [00:00<07:22,  4.52it/s]


KeyboardInterrupt: 